In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,LabelEncoder,MinMaxScaler,OneHotEncoder
import pickle as pkl
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from tensorflow.keras.models import load_model

Determine Optimal number of hidden layers

In [3]:
data=pd.read_csv("E:\DATA SETS\ANN\ANN_classification\data\Churn_Modelling.csv")

In [4]:
data

,conRowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [5]:
#load encoder and scaler pkl file 
model=load_model('model.h5')
with open('LabelEncoder_gender.pkl','rb') as file:
    LabelEncoder_gender=pkl.load(file)


with open('scaler.pkl','rb') as file:
    scaler=pkl.load(file)

with open('OneHotEncoder.pkl','rb') as file:
    LabelEncoder_geo=pkl.load(file)

In [6]:
data = data.drop(['conRowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()

data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')

geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()

geo_df=pd.DataFrame(geo_encoded,columns=LabelEncoder_geo.get_feature_names_out(['Geography']))

df = pd.concat([data.drop('Geography', axis=1), geo_df], axis=1)

X=df.drop(['Exited'],axis=1)
y=df['Exited']

In [10]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=51,test_size=0.30)

In [11]:
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [1]:
# df=Data.drop(["Surname","conRowNumber","CustomerId"],axis=1)
# LabelEncoder_gender=LabelEncoder()

# geo_encoded=LabelEncoder_geo.transform([df["Geography"]])
# geo_df=pd.DataFrame(geo_encoded,columns=LabelEncoder_geo.get_feature_names_out(['Geography']))
# input_data=pd.DataFrame([geo_df])
# input_data["Gender"]=LabelEncoder_gender.transform([input_data["Gender"]])
# input_data=pd.concat([input_data.drop("Geography",axis=1),geo_df],axis=1)
# X=Data.drop(['Exited'],axis=1)
# y=Data['Exited']

# X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=51,test_size=0.30)

# # input_data=input_data[['CreditScore','Gender','Age','Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember','EstimatedSalary','Geography_France','Geography_Germany','Geography_Spain']]
# X_train=scaler.fit_transform(X_train)
# X_test=scaler.transform(X_test)

# with open ("scaler.pkl","wb") as file:
#     pkl.dump(scaler, file)

# with open('LabelEncoder_gender.pkl','wb') as file:
#     pkl.dump(LabelEncoder_gender,file)

# with open('OneHotEncoder.pkl','wb') as file:
#     pkl.dump(LabelEncoder_geo,file)


In [9]:
#save encoder and scaler for later use

with open("scaler.pkl",'wb') as file:
    pkl.dump(scaler,file)

with open('LabelEncoder_gender','wb') as file:
    pkl.dump(label_encoder_gender, file)

with open('onehot_encoder_geo','wb') as file:
    pkl.dump(onehot_encoder_geo, file)

In [20]:
#define a function to create 
def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))
    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))

    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [25]:
#create keras classifier

model=KerasClassifier(neurons=32,layers=1,build_fn=create_model,verbose=0)

In [26]:
# Define hyperparameters to tune
param_grid = {
    'layers':[1,2],
    'batch_size': [10, 20, 40],
    'epochs': [10, 50],
    'neurons': [16, 32, 64]
}

In [27]:
grid_cv=GridSearchCV(estimator=model,param_grid=param_grid,n_jobs=-1,cv=4)
grid_result=grid_cv.fit(X_train,y_train)
print("best score %f using %s" % (grid_result.best_score_,grid_result.best_params_))

e:\projects\DecPro\venv\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


best score 0.862286 using {'batch_size': 40, 'epochs': 50, 'layers': 1, 'neurons': 32}
